<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 18.0 MB/s eta 0:00:00


In [2]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta

# Loading DataFrames



In [12]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-12-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [13]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [14]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2220299 17697 77401


# Data Preprocessing

In [15]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)
# Entry Date to date
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]
# dropping backlog invoices, Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]
#insert purchase indication column
df["purchase"] = 1

In [16]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [90]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')

  return df_pl

In [91]:
df_2 = sku_count(df_2)

<ipython-input-90-65d39e8a7ed9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')


In [10]:
# drop customers that only purchased 1 SKU
#df = df[df["sku_count"] > 1]

In [19]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [20]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [21]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,...,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,sku_count,bcn_id,sku_id
213,1545306,2022-11-02,NaN,2022FM11,15885514,AXIS SOLUTION (PRIVATE) LIMITED,Export Channel (DE),,CF55877,DT PRINT ZQ320 KIT LABEL SENSOR,...,5805,AIDC/PoS Printers,Mobile Receipt Printer,"Other (incl. AIDC/POS, V7)","10.713,30",41.0,1,20,497,49835
458,4422886,2022-11-03,NaN,2023FM02,44413224,BWG INFORMATIONSYSTEME GMBH,Business Channel,,CF89211,Z-SELECT 2000D REMOVABLE NS,...,5812,AIDC/PoS Printers,Label Printers Supplies,"Other (incl. AIDC/POS, V7)","1.393,00",140.0,1,111,1000,52759


In [22]:
users = df.bcn_id.unique()
skus = df.sku_id.unique()
print(len(users), len(skus))

13894 75643


In [23]:
df_2 = df[["bcn_id", "sku_id", "Entry Date", "Qty Shipped", "purchase"]]

In [24]:
grouped_df = df_2.groupby(["bcn_id", "sku_id"]).agg({
          "Qty Shipped":"sum",
          "purchase":"sum"}).reset_index()

In [25]:
# create binary column
grouped_df["purch_bin"] = 1

In [26]:
grouped_df.head(2)

,bcn_id,sku_id,Qty Shipped,purchase,purch_bin
0,0,928,1.0,1,1
1,0,3422,1.0,1,1


In [27]:
grouped_df_2 = grouped_df.copy()

## Negative Sampling

In [28]:
def negative_sampling(df_pl, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_pl = pd.concat([df_pl, df_neg], ignore_index=True)

    return df_pl

In [29]:
neg_grouped_df = negative_sampling(grouped_df_2, users, skus, skus, n_neg=100)

In [30]:
neg_grouped_df = neg_grouped_df.fillna(0)

In [31]:
print(len(grouped_df), len(neg_grouped_df))

544417 1933817


## ALS Model param tuning

In [33]:
import implicit
from implicit.gpu.als import AlternatingLeastSquares as ALS
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split

In [34]:
csr_qty_matrix = sparse.csr_matrix((grouped_df['Qty Shipped'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_freq_matrix = sparse.csr_matrix((grouped_df['purchase'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_bin_matrix = sparse.csr_matrix((grouped_df['purch_bin'], (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [35]:
neg_csr_qty_matrix = sparse.csr_matrix((neg_grouped_df['Qty Shipped'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_freq_matrix = sparse.csr_matrix((neg_grouped_df['purchase'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_bin_matrix = sparse.csr_matrix((neg_grouped_df['purch_bin'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))

In [36]:
csr_qty_matrix_train, csr_qty_matrix_test = leave_k_out_split(csr_qty_matrix, K=1, random_state=None)
csr_freq_matrix_train, csr_freq_matrix_test = leave_k_out_split(csr_freq_matrix, K=1, random_state=None)
csr_bin_matrix_train, csr_bin_matrix_test = leave_k_out_split(csr_bin_matrix, K=1, random_state=None)

In [37]:
csr_qty_matrix_train_2, csr_qty_matrix_test_2 = train_test_split(csr_qty_matrix, train_percentage=0.9, random_state=None)
csr_freq_matrix_train_2, csr_freq_matrix_test_2 = train_test_split(csr_freq_matrix, train_percentage=0.9,  random_state=None)
csr_bin_matrix_train_2, csr_bin_matrix_test_2 = train_test_split(csr_bin_matrix, train_percentage=0.9, random_state=None)

In [38]:
neg_csr_qty_matrix_train, neg_csr_qty_matrix_test = leave_k_out_split(neg_csr_qty_matrix, K=1, random_state=None)
neg_csr_freq_matrix_train, neg_csr_freq_matrix_test = leave_k_out_split(neg_csr_freq_matrix, K=1, random_state=None)
neg_csr_bin_matrix_train, neg_csr_bin_matrix_test = leave_k_out_split(neg_csr_bin_matrix, K=1, random_state=None)

In [50]:
model = ALS()

  0%|          | 0/15 [00:00<?, ?it/s]

In [67]:
model.fit(csr_freq_matrix_train_2)

  0%|          | 0/15 [00:00<?, ?it/s]

In [72]:
auc10 = AUC_at_k(model, csr_freq_matrix_train_2, csr_freq_matrix_test_2, K=10, num_threads=10)
auc10

  0%|          | 0/6944 [00:00<?, ?it/s]

0.6089061782736811

## Hyperparametertuning


In [76]:
from IPython.utils.sysinfo import num_cpus
from sklearn.model_selection import ParameterGrid

# Grid of hyperparameters to search
param_grid = {
    'factors': [100, 150, 200],
    'iterations':[10, 15, 20],
    'alpha': [20, 30, 40],
    'regularization': [0.01, 0.1]
}

best_auc = -np.inf
best_params = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model.fit(csr_qty_matrix_train, show_progress=False)

    auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10 > best_auc:
        best_auc = auc10
        best_params = params

# Print the best parameters and AUC
print("Best parameters:", best_params)
print("Best AUC:", best_auc)

Best parameters: {'alpha': 20, 'factors': 200, 'iterations': 20, 'regularization': 0.01}
Best AUC: 0.5875023428285527


In [77]:
from IPython.utils.sysinfo import num_cpus
from sklearn.model_selection import ParameterGrid

# Grid of hyperparameters to search
param_grid = {
    'factors': [100, 150, 200],
    'iterations':[10, 15, 20],
    'alpha': [20, 30, 40],
    'regularization': [0.01, 0.1]
}

best_auc_2 = -np.inf
best_params_2 = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model_2 = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model_2.fit(csr_freq_matrix_train, show_progress=False)

    auc10 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10 > best_auc:
        best_auc = auc10
        best_params = params

# Print the best parameters and AUC
print("Best parameters:", best_params)
print("Best AUC:", best_auc)

Best parameters: {'alpha': 40, 'factors': 200, 'iterations': 10, 'regularization': 0.01}
Best AUC: 0.5915934182245448


In [79]:
model = ALS(**best_params)
model.fit(csr_qty_matrix_train)
auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.5830489258289672; Prec@10: 0.1662178956089478; Map@10: 0.0914435551215793; Ndcg@10: 0.10920116810956147


In [81]:
model_2 = ALS(**best_params_2)
model_2.fit(csr_freq_matrix_train)
auc10 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.5733646566446018; Prec@10: 0.14685169842584922; Map@10: 0.0759271725121976; Ndcg@10: 0.09270683467765381


## Recommending with ALS

In [92]:
df_2

,bcn_id,sku_id,Entry Date,Qty Shipped,purchase,sku_count
213,497,49835,2022-11-02,41.0,1,20
458,1000,52759,2022-11-03,140.0,1,111
1425,2139,53004,2022-11-08,1.0,1,9086
2112,8216,33756,2022-11-09,5.0,1,8
3786,3641,35337,2022-11-14,1.0,1,372
...,...,...,...,...,...,...
2220294,11119,46629,2022-12-28,21.0,1,282
2220295,1966,24994,2023-06-06,1.0,1,251
2220296,1966,73802,2023-06-06,1.0,1,251
2220297,11681,29253,2022-12-19,2.0,1,120


In [194]:
users_less_20 = df_2[(df_2.sku_count <= 20) & (df_2.sku_count >= 5) ]["bcn_id"].unique()
users_more_20 = df_2[df_2.sku_count > 20]["bcn_id"].unique()

In [214]:
 # Assuming you want recommendations for user with ID 10
import random
user_id = random.choice(users_less_20)
user_id_2 = random.choice(users_more_20)
print(user_id)
print(user_id_2)

5138
7783


In [215]:
user_purchases = pd.DataFrame(df_2[(df.bcn_id == user_id)])
user_purchases = user_purchases.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases = user_purchases.sort_values(by="Qty Shipped", ascending=False)
user_purchases = user_purchases.merge(sku_list, on="sku_id", how="left")
user_purchases

,bcn_id,sku_id,Qty Shipped,purchase,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,5138,65699,5.0,2,HDMI TO VGA ADAPTER WHITE,Monitor Cables & Adapters,Cables
1,5138,17913,2.0,1,LC-223M INK CARTRIDGE MAGENDA,Ink,Consumables
2,5138,17914,2.0,1,LC-223Y YELLOW INK CARTRIDGE,Ink,Consumables
3,5138,33576,2.0,1,2D BARCODE HAND SCANNER QR CODE,Handheld Barcode Scanner,AIDC/PoS Barcode Scanners
4,5138,65764,2.0,1,V7 TONER KYOCERA TK-170 BK,Toner Color Laser,Consumables
5,5138,17685,1.0,1,DR-2200 DRUM UNIT 12000 PAGES,"Laser Fusing Unit, Coating, Staple Cart, Drum",Consumables
6,5138,17917,1.0,1,LC-223C INK CARTRIDGE CYAN,Ink,Consumables
7,5138,18074,1.0,1,TINTE BLACK 400 PAGES,Ink,Consumables
8,5138,37751,1.0,1,XEROX C310 BLACK HIGH CAPACITY,Toner Color Laser,Consumables
9,5138,37752,1.0,1,XEROX C310 CYAN HIGH CAPACITY,Toner Color Laser,Consumables


In [216]:
user_purchases_2 = pd.DataFrame(df_2[(df.bcn_id == user_id_2)])
user_purchases_2 = user_purchases_2.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases_2 = user_purchases_2.sort_values(by="Qty Shipped", ascending=False)
user_purchases_2 = user_purchases_2.merge(sku_list, on="sku_id", how="left")
user_purchases_2

,bcn_id,sku_id,Qty Shipped,purchase,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,7783,68142,1962.0,6,3M 15.6W PRIVACY FILTER,Portable/Notebook Accs,Computer Systems
1,7783,52453,1530.0,5,4Y ONSITE FROM 1Y COURIER/CARRY,Services And Warranties,Computer Systems
2,7783,35350,1500.0,5,TP USB-C WIRED MOUSE,Mouse,Input Devices
3,7783,68475,1440.0,6,LENOVO 15.6 BASIC TOPLOADER,Carrying Cases,Accessories
4,7783,44266,1300.0,8,TP L15 G3 I5-1235U 16GB,Portable/Notebook Computers,Computer Systems
5,7783,28197,1300.0,3,TP UNIVERSAL USB-C DOCK,"Port Replicator, Docking Station",Accessories
6,7783,68203,1200.0,2,LENOVO LEGION GAMING,Wrist/Mouse Pads,Office Equipment
7,7783,4184,1050.0,2,KEYBOARD K120 F/ BUSINESS,Keyboards & Keypads,Input Devices
8,7783,37854,1050.0,2,KENSINGTON NS MASTERKEY,Pc Security Devices,Security
9,7783,68300,850.0,2,KENSINGTON MS DS 2.0,Pc Security Devices,Security


## Recommendations

In [217]:
sku_list = df[["sku_id", "Product Descr1", "ProductGroupDescription", "ProductGroupMasterDescription"]].drop_duplicates()
sku_list

,sku_id,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
213,49835,DT PRINT ZQ320 KIT LABEL SENSOR,Mobile Receipt Printer,AIDC/PoS Printers
458,52759,Z-SELECT 2000D REMOVABLE NS,Label Printers Supplies,AIDC/PoS Printers
1425,53004,K120G2-R I5-1135G7 12.5IN W/CAM,Tablets (Slate Style),Computer Systems
2112,33756,Z-PERF 1000D 80 RECEIPT 101.6MM,Label Printers Supplies,AIDC/PoS Printers
3786,35337,JABRA EVOLVE2 75 LINK380A MS,Bluetooth Headsets,Mobility
...,...,...,...,...
2220237,43142,ARUBA 3810M 4SFP+ MODULE,Module,Communications & Networking
2220238,43144,ARUBA CNTRLR PER AP ENT LIC,Network Management & Utility Software,Softwares
2220239,9991,HP 4.5MM + USB-C DOCK ADAPT. G2,"Port Replicator, Docking Station",Accessories
2220246,33944,MC:WLANX BT SS 2DERI 38KY 2.5X,Handheld Terminal,AIDC Mobile Terminals


#### Model 1

In [218]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,17912.0,0.032472,LC-223BK BLACK INK CARTRIDGE,Ink,Consumables
1,18077.0,0.030752,TINTE YELLOW 400 PAGES,Ink,Consumables
2,18012.0,0.028021,LC-3219XLY INKCARTRIDGE YELLOW,Ink,Consumables
3,18006.0,0.027825,LC-3219XLM INK CARTRIDGE MAGEN,Ink,Consumables
4,17830.0,0.026704,LC-123BK F. MFC-J4510DW,Ink,Consumables
5,18075.0,0.026525,TINTE CYAN 400 PAGES,Ink,Consumables
6,42691.0,0.025560,LC-422XLY INK YELLOW F.,Ink,Consumables
7,42688.0,0.025404,LC-422XLBK INK BLACK F.,Ink,Consumables
8,18008.0,0.025333,LC-3217C INK CARTRIDGE CYAN,Ink,Consumables
9,18004.0,0.024815,LC-3217M INK CARTRIDGE MAGENTA,Ink,Consumables


In [219]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,16519.0,1.030952,WORRY FREE 9 ADVANCED,"Virus Detection, Antivirus Software",Softwares
1,16602.0,0.979898,WORRY FREE 9 STANDARD,"Virus Detection, Antivirus Software",Softwares
2,67494.0,0.842706,LANCOM ADVANCED VPN CLIENT,Network Management & Utility Software,Softwares
3,18529.0,0.801257,MICROSAVER 2.0 KEYED LOCK,Pc Security Devices,Security
4,12107.0,0.739316,MOBILE LITE PLUS USB 3.1,Flash Card Reader/Writers,Hard Drives & Optical Drives
5,68572.0,0.730734,3Y PREMIER SUPPORT UPGRADE FROM,Services And Warranties,Computer Systems
6,32904.0,0.727062,TP UNIVERS.THUNDERBOLT 4 DOCK,"Port Replicator, Docking Station",Accessories
7,68481.0,0.725453,EPAC MANAGED MS AUTOPILOT REGIS,Services And Warranties,Computer Systems
8,34240.0,0.695048,TRELLIX PROTECT STANDARD,Other Security Software,Softwares
9,68296.0,0.678717,TP HYBRID USB-C W. USB-A DOCK,"Port Replicator, Docking Station",Accessories


#### Model 2

In [220]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model_2.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,18082.0,0.021093,TN-2420 TONER 3000 PAGES,Toner Mono Laser,Consumables
1,17940.0,0.019903,LC-223 BLISTER,Ink,Consumables
2,18129.0,0.019781,TN-243CMYK MULTIPACK 4 1000P,Toner Color Laser,Consumables
3,17683.0,0.019569,TN-2220 TONER 2600 PAGES,Toner Mono Laser,Consumables
4,18005.0,0.019204,LC-3219XL BLISTER COLOR INK SET,Ribbon,Consumables
5,18128.0,0.019110,TN-247BK JUMBO TONER BLACK 3K P,Toner Color Laser,Consumables
6,18006.0,0.018559,LC-3219XLM INK CARTRIDGE MAGEN,Ink,Consumables
7,18169.0,0.018158,LC-3217 VALDR INK SET,Printer/Plotter Accs,Printers/AIO/Copiers/Fax
8,18042.0,0.018127,TN-421M TONER FOR BC4,Toner Color Laser,Consumables
9,18084.0,0.017940,DR-2400 DRUM UNIT 12000,"Laser Fusing Unit, Coating, Staple Cart, Drum",Consumables


In [221]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model_2.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,68572.0,0.205037,3Y PREMIER SUPPORT UPGRADE FROM,Services And Warranties,Computer Systems
1,68296.0,0.199243,TP HYBRID USB-C W. USB-A DOCK,"Port Replicator, Docking Station",Accessories
2,31095.0,0.177343,TP THUNDERBOLT 4 DOCK G2,"Port Replicator, Docking Station",Accessories
3,68257.0,0.169655,LENOVO 65W STAND. AC ADAPTER,Notebook Battery & Ac Adapter,Computer Systems
4,32544.0,0.166837,TP ESSENT.PLUS 16.0 TOPLOADER,Carrying Cases,Accessories
5,68159.0,0.160930,EPAC 3 YRS ONSITE NBD,Services And Warranties,Computer Systems
6,68576.0,0.160226,T24I-2L 23.8 FHD IPS 16:9,Lcd Monitor,Display
7,28669.0,0.160157,EPAC WARRANTY 3Y PREMIER SUPPOR,Services And Warranties,Computer Systems
8,32904.0,0.159421,TP UNIVERS.THUNDERBOLT 4 DOCK,"Port Replicator, Docking Station",Accessories
9,37067.0,0.140148,LENOVO PERFORM. FHD WEBCAM,Webcamera,Multimedia & Games
